# Nalantis to eFLINT

This notebook converts the intermediate Nalantis structure to eFLINT code.

In [3]:
import json

In [7]:
extracted = [
    {'legalrule': 'Rule1', 'dutyholder': 'remote_pilot', 'actor': 'drone', 'state': 'in-flight', 'preconditions': [{'attribute': 'mtom', 'value': 3, 'constraint': '>=', 'unit': 'kg'}, {'attribute': 'location', 'value': ['Spain', 'Belgium'], 'constraint': 'OR'}, {'attribute': 'time', 'value': 'Night', 'constraint': '=='}], 'duties': [{'attribute': 'kept_at_safe_distance', 'constraint': '==', 'value': 'TRUE'}, {'attribute': 'flown_over_assemblies', 'constraint': '==', 'value': 'FALSE'}]},
    {'legalrule': 'Rule2', 'dutyholder': 'remote_pilot', 'actor': 'remote_pilot', 'state': 'in-flight-2', 'preconditions': [], 'duties': [{'attribute': 'complies_with_minimum_age', 'constraint': '>=', 'value': 16}]}
]

In [8]:
print(json.dumps(extracted, indent=4))

[
    {
        "legalrule": "Rule1",
        "dutyholder": "remote_pilot",
        "actor": "drone",
        "state": "in-flight",
        "preconditions": [
            {
                "attribute": "mtom",
                "value": 3,
                "constraint": ">=",
                "unit": "kg"
            },
            {
                "attribute": "location",
                "value": [
                    "Spain",
                    "Belgium"
                ],
                "constraint": "OR"
            },
            {
                "attribute": "time",
                "value": "Night",
                "constraint": "=="
            }
        ],
        "duties": [
            {
                "attribute": "kept_at_safe_distance",
                "constraint": "==",
                "value": "TRUE"
            },
            {
                "attribute": "flown_over_assemblies",
                "constraint": "==",
                "value": "FALSE"
            }
     

In [9]:
def get_value(value):
    # Retrieve the value of a variable
    if type(value) == str:
        return '"' + str(value) + '"'
    return str(value)

def make_condition(condition):
    # Build a condition
    return condition['attribute'] + ' ' + condition['constraint'] + ' ' + get_value(condition['value'])

def get_related(rule):
    # Get related rules
    related = []
    for condition in rule['preconditions']:
        related.append(condition['attribute'])
    return related

def make_conditions(rule):
    # Make conditions
    conditions = []
    for condition in rule['preconditions']:
        if condition['constraint'] == 'OR':
            subconditions = []
            for value in condition['value']:
                subcondition = make_condition({**condition, **{
                    'constraint': '==',
                    'value': value
                }})
                subconditions.append(subcondition)
            conditions.append('(' + ' || '.join(subconditions) + ')')
        else:
            conditions.append(make_condition(condition))
    return conditions

def make_duty(rule):
    # Make a duty
    output = ''
    output += "Duty duty-during-" + rule['state'] + "\n"
    output += "  Holder " + rule['dutyholder'] + "\n"
    output += "  Claimant " + rule['dutyholder'] + "\n"
    output += "  Violated when is-" + rule['state'] + "(" + rule['dutyholder'] + ") && "
    duty_violations = []
    for duty in rule['duties']:
        duty_violations.append(("!" if duty['value'] == 'TRUE' else '') + duty['attribute'] + '(' + rule['dutyholder'] + ')')
    output += "(" + ' || '.join(duty_violations) + ")"
    return output

def make_start_act(rule):
    # Make the start act
    output = ''
    conditions = make_conditions(rule)
    output += 'Act start-' + rule['state'] + "\n"
    output += '  Actor ' + rule['dutyholder'] + "\n"
    output += '  Recipient ' + rule['dutyholder'] + "\n"
    output += '  Related to ' + ', '.join(get_related(rule)) + "\n"
    output += '  Conditioned by !is-' + rule['state'] + '(' + rule['dutyholder'] + ') && ' + ' && '.join(conditions) + "\n"
    output += '  Creates is-' + rule['state'] + '(' + rule['dutyholder'] + '), duty-during-' + rule['state'] + '(' + rule['dutyholder'] + ', ' + rule['dutyholder'] + ')' + "\n"
    return output

def make_stop_act(rule):
    # Make the stop act
    output = ''
    output += 'Act stop-' + rule['state'] + "\n"
    output += '  Actor ' + rule['dutyholder'] + "\n"
    output += '  Recipient ' + rule['dutyholder'] + "\n"
    output += '  Conditioned by is-' + rule['state'] + '(' + rule['dutyholder'] + ')' + "\n"
    output += '  Terminates is-' + rule['state'] + '(' + rule['dutyholder'] + '), duty-during-' + rule['state'] + '(' + rule['dutyholder'] + ', ' + rule['dutyholder'] + ')' + "\n"
    return output

def get_acts(rule):
    # Retrieve possible acts
    output = []
    output.append('start-' + rule['state'] + '.')
    output.append('stop-' + rule['state'] + '.')
    return output

def extend_domains(rule, current_domains):
    # Extent domains
    domains = current_domains
    
    # @TODO, assumption that Alice and Bob are dutyholders
    if rule['dutyholder'] not in current_domains:
        domains[rule['dutyholder']] = {'Alice', 'Bob'}
        
    for condition in rule['preconditions']:
        if type(condition['value']) == list:
            values = condition['value']
        else:
            values = [condition['value']]
        for value in values:
            if condition['attribute'] not in current_domains:
                current_domains[condition['attribute']] = set()
            current_domains[condition['attribute']].add(value)
    return domains

def make_domains(domains):
    # Make domains
    output = ''
    for domain, domain_values in domains.items():
        domain_type = "str"
        if all(type(domain_value) == int for domain_value in domain_values):
            domain_type = "int"
        if domain_type == "int":
            output += 'Fact ' + domain + ' Identified by ' + str(min(domain_values)) + '..' + str(max(domain_values)) + "\n"
        else:
            output += 'Fact ' + domain + ' Identified by ' + ', '.join(domain_values) + '\n'
    return output

def make_facts(rule):
    # Make the facts
    output = []
    output.append('Fact ' + rule['dutyholder'])
    for condition in rule['preconditions']:
        suffix = ''
        if type(condition['value']) == int:
            suffix = ' Identified by Int'
        output.append('Fact ' + condition['attribute'] + suffix)
    return output

def make_relations(rule):
    # Make the relations
    output = []
    output.append('Fact is-' + rule['state'] + ' Identified by ' + rule['dutyholder'])
    for duty in rule['duties']:
        output.append('Fact ' + duty['attribute'] + ' Identified by ' + rule['dutyholder'])
    return output

# Extract information
output = []
acts = []
domains = {}
facts = []
relations = []
for rule in extracted:
    acts.extend(get_acts(rule))
    facts.extend(make_facts(rule))
    relations.extend(make_relations(rule))
    domains = extend_domains(rule, domains)
facts = list(set(facts))
relations = list(set(relations))

# Build the output
output.append("#")
output.append("\n".join(facts))
output.append("\n".join(relations))
for rule in extracted:
    output.append(make_duty(rule))
    output.append(make_start_act(rule))
    output.append(make_stop_act(rule))
output.append("##")
output.append(make_domains(domains))
output.append("###")
acts = list(set(acts))
output.append("\n".join(acts))
output.append("####")
output = "\n\n".join(output)

# Add a demo scenario
output += "\n\n" + """// Example
+kept_at_safe_distance(Alice).
start-in-flight(remote_pilote=Alice, mtom=3, location=Belgium, time=Night).
?is-in-flight(Alice)."""
    
# Print the output
print(output)

# Write the output to file
with open('outputv2.eflint', 'w') as out_file:
    out_file.write(output)

#

Fact mtom Identified by Int
Fact location
Fact time
Fact remote_pilot

Fact kept_at_safe_distance Identified by remote_pilot
Fact is-in-flight Identified by remote_pilot
Fact is-in-flight-2 Identified by remote_pilot
Fact complies_with_minimum_age Identified by remote_pilot
Fact flown_over_assemblies Identified by remote_pilot

Duty duty-during-in-flight
  Holder remote_pilot
  Claimant remote_pilot
  Violated when is-in-flight(remote_pilot) && (!kept_at_safe_distance(remote_pilot) || flown_over_assemblies(remote_pilot))

Act start-in-flight
  Actor remote_pilot
  Recipient remote_pilot
  Related to mtom, location, time
  Conditioned by !is-in-flight(remote_pilot) && mtom >= 3 && (location == "Spain" || location == "Belgium") && time == "Night"
  Creates is-in-flight(remote_pilot), duty-during-in-flight(remote_pilot, remote_pilot)


Act stop-in-flight
  Actor remote_pilot
  Recipient remote_pilot
  Conditioned by is-in-flight(remote_pilot)
  Terminates is-in-flight(remote_pilot), du